<a href="https://colab.research.google.com/github/Pavitra-khare/DA6401_ASS_2/blob/main/DA6401_ASS_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [1]:
# Import necessary libraries
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import torch.nn as nn
import torch.optim as optim
import numpy as np
import wandb

In [1]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 3628-pavitrakhare (3628-pavitrakhare-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split

def split_dataset(dataset, validation_ratio=0.1):
    """
    Divides a dataset into training and validation portions.

    Args:
        dataset (torch.utils.data.Dataset): Dataset to be divided.
        validation_ratio (float): Proportion of the dataset to be used for validation. Default is 0.1.

    Returns:
        dict: Contains 'train' and 'val' datasets as Subset objects.
    """
    total_samples = len(dataset)
    indices = list(range(total_samples))
    val_indices, train_indices = train_test_split(indices, test_size=1 - validation_ratio)

    subsets = {
        "train": Subset(dataset, train_indices),
        "val": Subset(dataset, val_indices)
    }

    return subsets


In [ ]:
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import torch

def get_data_loaders():
    """
    Generate training and validation data loaders with various data augmentations.

    Returns:
        tuple: Training and validation DataLoader objects.
    """
    # Normalization stats and output image size
    norm_mean = (0.5, 0.5, 0.5)
    norm_std = (0.5, 0.5, 0.5)
    image_size = 32

    # Define multiple transformation pipelines
    basic_transform = transforms.Compose([
        transforms.RandomResizedCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize(norm_mean, norm_std)
    ])

    horizontal_aug = transforms.Compose([
        transforms.RandomResizedCrop(image_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])

    vertical_aug = transforms.Compose([
        transforms.RandomResizedCrop(image_size),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor()
    ])

    invert_aug = transforms.Compose([
        transforms.RandomResizedCrop(image_size),
        transforms.RandomInvert(),
        transforms.ToTensor()
    ])

    # Path to the dataset directory
    data_path = "/content/drive/MyDrive/dl-assigment-2/inaturalist_12K/train"

    # Create multiple datasets using different augmentations
    base_dataset = torchvision.datasets.ImageFolder(root=data_path, transform=basic_transform)
    horiz_dataset = torchvision.datasets.ImageFolder(root=data_path, transform=horizontal_aug)
    vert_dataset = torchvision.datasets.ImageFolder(root=data_path, transform=vertical_aug)
    invert_dataset = torchvision.datasets.ImageFolder(root=data_path, transform=invert_aug)

    # Combine all datasets into a single dataset
    combined_dataset = base_dataset + horiz_dataset + vert_dataset + invert_dataset

    # Split the dataset into training and validation portions
    split_data = split_dataset(combined_dataset)

    # Prepare data loaders
    train_loader = DataLoader(split_data['train'], batch_size=128, shuffle=True)
    val_loader = DataLoader(split_data['val'], batch_size=128, shuffle=False)

    return train_loader, val_loader
